# Collaborative Filtering project by Paige McKenzie

Includes code to perform analysis discussed in my [blog post]().

[Dataset](https://www.kaggle.com/azathoth42/myanimelist/version/9) available here.

https://realpython.com/build-recommendation-engine-collaborative-filtering/

In [ ]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
%pylab inline

In [ ]:
# import data
shows = pd.read_csv('anime_filtered.csv', index_col='anime_id', usecols=['title', 'anime_id'])
reviews = pd.read_csv('animelists_filtered.csv', nrows=400000, usecols=['username', 'anime_id', 'my_score'])

In [ ]:
# downsample to a complete set of reviews for a subset of shows
reviews = reviews[reviews['anime_id'].isin(reviews['anime_id'].unique()[:-1])]

In [ ]:
# pivot for one row per user, and column per anime
reviews = pd.pivot_table(reviews, index='username', columns='anime_id', values='my_score', aggfunc=max)

reviews.head()

In [ ]:
# define our target show
target_cols = reviews.columns[-3:]

In [ ]:
# only keep users who have rated at least one target show
# also only keep users who have rated at least one other show (users we have some information about)
reviews = reviews.loc[reviews[target_cols].notna().max(axis=1) 
                      & (reviews.drop(target_cols, axis=1).notna().sum(axis=1)>0)]

# sample for data scale
reviews = reviews.sample(frac=.6)
reviews.shape

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(311)
plt.title("Distribution of reviews for '{}'".format(shows.loc[target_cols[0], 'title']))
plt.hist(reviews[target_cols[0]].dropna())
plt.axvline(reviews[target_cols[0]].mean(), color='purple', ls='--')

pyplot.subplot(312)
plt.title("Distribution of reviews for '{}'".format(shows.loc[target_cols[1], 'title']))
plt.hist(reviews[target_cols[1]].dropna())
plt.axvline(reviews[target_cols[1]].mean(), color='purple', ls='--')

pyplot.subplot(313)
plt.title("Distribution of reviews for '{}'".format(shows.loc[target_cols[2], 'title']))
plt.hist(reviews[target_cols[2]].dropna())
plt.axvline(reviews[target_cols[2]].mean(), color='purple', ls='--')

pyplot.show()

We can see that the majority of people who bother to rate a show do so to assert their dislike of it (hense the spike at zero). Everybody else offered a little more granularity, with most people really liking it (a score of 10) with a tail towards zero. Clearly, we would never want to recommend this show to someone who we think is going to hate it.

For this exercise, we'll split the dataset into two groups:

1. Train (the "known") - users who have scored the show we'll recommend, and whose scores we'll use to model
2. Test (the "unknown") - users who have scored the show we'll recommend, but whose scores we'll ignore and only use at the end, for measuring how well we targeted the subset that would enjoy the show

Our goal for this project is to recommend each show to some of the unknown users, with the goal of recommending each show to the subset of users that will enjoy the show most (rate it highest).

Let's also assume we will only suggest the show to the top `target_frac` of unknown users (in this case, 20%), factoring in how popular the show is (based on how many users in the train set have rated it). More succinctly, we will recommend the show to more people if the show has a lot of ratings in the train set, and recommend to fewer people if it doesn't have as many ratings, but our goal of targeting the top 20% will stay the same despite the show's viewership numbers.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size=.3, random_state=1)

del reviews

In [ ]:
target_frac = 0.2

target_sizes = [int(len(test)*train[target_col].notna().mean()*target_frac) for target_col in target_cols]

In [ ]:
# baseline (just random sample)
baselines = []

for target_col, size in zip(target_cols, target_sizes):
    score = test.loc[test[target_col].notna(), target_col].sample(size, random_state=1).mean()
    baselines.append(score)
    print("Average rating (random sample) when recommending '{}' to {} users: {}".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2)))

In [ ]:
# baseline (sort by average rating user gives other shows)

for target_col, size, baseline in zip(target_cols, target_sizes, baselines):
    score = test.loc[test.drop(target_col, axis=1).mean(axis=1)[test[target_col].notna()].sort_values(ascending=False).head(size).index,
                        target_col].mean()
    print("Average rating (highest user-average on other shows) when recommending '{}' to {} users: {}\n\t- Lift of {} over baseline\n".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2),
        round(score/baseline, 2)))

All we've really managed to do here is target "happier" users, or perhaps those with lower standards - however, it has produced a noticeable lift in the scores we get when recommending each show. Can we do better, by using each user's reviews on other shows as a basis for their enjoyment of these shows?

## Collaborative Filtering

In [ ]:
train_targets = train[target_cols]
test_targets = test[target_cols]

train = train.drop(target_cols, axis=1)
test = test.drop(target_cols, axis=1)

In [ ]:
# zero-center reviews, saving the average per user
train_avg = train.mean(axis=1)
test_avg = test.mean(axis=1)

train = train.apply(lambda col:col-train_avg)
test = test.apply(lambda col:col-test_avg)

train_targets = train_targets.apply(lambda col:col-train_avg)
test_targets = test_targets.apply(lambda col:col-test_avg)

In [ ]:
# find inter-user similarity (ignoring our target columns)
from sklearn.metrics.pairwise import cosine_similarity

sim = pd.DataFrame(cosine_similarity(train.fillna(0), test.fillna(0)), 
                   index=train.index, columns=test.index)

In [ ]:
# one row per known user, one column per unknown user
sim.shape

In [ ]:
def most_similar_user(col):
    # find most similar known user
    user = col.idxmax()
    
    # return that user's adjustment for this show
    ## if user is similar at all
    return train_targets.loc[user, target_col] if col.max()>0 else 0
    
for target_col, size, baseline in zip(target_cols, target_sizes, baselines):
    # actual ratings for first target show (for those users in the test set who rated it)
    actual = (test_targets[target_col]+test_avg).dropna()
    
    # get the most similar user's adjustment on this show, 
    ## then apply that adjustment to the unknown user's avg score
    pred = sim.loc[train_targets[target_col].notna(), actual.index].apply(most_similar_user)+test_avg.reindex(actual.index)
    # adjust impossible scores
    pred[pred<0] = 0
    pred[pred>10] = 10

    score = actual[pred.sort_values(ascending=False).head(size).index].mean()
    
    print("Average rating (single most-similar user) when recommending '{}' to {} users: {}\n\t- Lift of {} over baseline\n".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2),
        round(score/baseline, 2)))

Not bad, but not as good as simply targeting "happy" users - perhaps because we're only using a single "most similar" user right now. Let's expand our criteria.

In [ ]:
def n_most_similar_users(col, n=10):
    # find most similar known user
    users = col.sort_values(ascending=False).head(n).index
    users = users[col[users]>0]
    
    if len(users)==0:
        # no similar users, no adjustment
        return 0
    
    # return the average adjustment across all similar users for this show
    return train_targets.loc[users, target_col].mean()
    
for target_col, size, baseline in zip(target_cols, target_sizes, baselines):
    # actual ratings for first target show (for those users in the test set who rated it)
    actual = (test_targets[target_col]+test_avg).dropna()
    
    # get the most similar users' adjustments on this show, 
    ## then apply that adjustment to the unknown user's avg score
    pred = sim.loc[train_targets[target_col].notna(), actual.index].apply(n_most_similar_users)+test_avg.reindex(actual.index)
    # adjust impossible scores
    pred[pred<0] = 0
    pred[pred>10] = 10

    score = actual[pred.sort_values(ascending=False).head(size).index].mean()
    
    print("Average rating (n=10 most-similar users) when recommending '{}' to {} users: {}\n\t- Lift of {} over baseline\n".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2),
        round(score/baseline, 2)))

**Wisdom of the herd!** You love to see it.

So what's next? Right now we're factoring in only the n-most similar users. But could it be possible that we could learn just as much from the most *dissimilar* users and moving in the opposite direction? Let's test it out.

In [ ]:
def least_similar_user(col):
    # find most similar known user
    user = col.idxmin()
    
    # return that user's adjustment for this show
    ## if user is similar at all
    return train_targets.loc[user, target_col] if col.min()<0 else 0
    
for target_col, size, baseline in zip(target_cols, target_sizes, baselines):
    # actual ratings for first target show (for those users in the test set who rated it)
    actual = (test_targets[target_col]+test_avg).dropna()
    
    # get the OPPOSITE of the LEAST similar user's adjustment on this show, 
    ## then apply that adjustment to the unknown user's avg score
    pred = -1*sim.loc[train_targets[target_col].notna(), actual.index].apply(most_similar_user)+test_avg.reindex(actual.index)
    # adjust impossible scores
    pred[pred<0] = 0
    pred[pred>10] = 10

    score = actual[pred.sort_values(ascending=False).head(size).index].mean()
    
    print("Average rating (single least-similar user) when recommending '{}' to {} users: {}\n\t- Lift of {} over baseline\n".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2),
        round(score/baseline, 2)))

Not as good as the most similar user method, but interestingly still some good information here. Our intuition was right. Let's see if the herd mentality works here as well.

In [ ]:
def n_least_similar_users(col, n=10):
    # find most similar known user
    users = col.sort_values(ascending=True).head(n).index
    users = users[col[users]<0]
    
    if len(users)==0:
        # no similar users, no adjustment
        return 0
    
    # return the average adjustment across all similar users for this show
    return train_targets.loc[users, target_col].mean()
    
for target_col, size, baseline in zip(target_cols, target_sizes, baselines):
    # actual ratings for first target show (for those users in the test set who rated it)
    actual = (test_targets[target_col]+test_avg).dropna()
    
    # get the OPPOSITE of the LEAST similar users adjustments on this show, 
    ## then apply that adjustment to the unknown user's avg score
    pred = -1*sim.loc[train_targets[target_col].notna(), actual.index].apply(n_most_similar_users)+test_avg.reindex(actual.index)
    # adjust impossible scores
    pred[pred<0] = 0
    pred[pred>10] = 10

    score = actual[pred.sort_values(ascending=False).head(size).index].mean()
    
    print("Average rating (n=10 least-similar users) when recommending '{}' to {} users: {}\n\t- Lift of {} over baseline\n".format(shows.loc[target_col, 'title'],
        size,
        round(score, 2),
        round(score/baseline, 2)))

Almost as good as the herd approach for similar users! 

What else can we try? Maybe a weighted average, rather than a straight average.

In [ ]:
def weighted_similarity(col, qnt):
    sim_subset = col.sort_values(ascending=False).iloc[1:qnt+1]
    return (views[target_col].reindex(sim_subset.index)*sim_subset).sum()/sim_subset.sum()

rating = sim.apply(lambda col:weighted_similarity(col, qnt=10)).fillna(0)

In [ ]:
sqrt(mean_squared_error((views[target_col]+avg_ratings)[views[target_col].notna()], 
                        (rating+avg_ratings)[views[target_col].notna()]))

## Alternate targets

In [ ]:
users = pd.read_csv('users_filtered.csv', index_col='username')
users = users[users.index.isin(reviews.index)]

### Linear regression prediction

In [ ]:
from sklearn.linear_model import LinearRegression

lin_mod = LinearRegression()

lin_mod.fit(train.drop(target_col, axis=1).apply(lambda col:col.fillna(col.median())).values, train[target_col])

lin_pred = lin_mod.predict(test.drop(target_col, axis=1).apply(lambda col:col.fillna(train[col.name].median())).values)

In [ ]:
# benchmark purely random guess
mean = pd.Series(lin_pred, index=test.index).sort_values(ascending=False).head(int(len(test)*target_frac)).mean()
print("Linear regression achieves an average score of {} for a lift of {}".format(round(mean, 2), 
                                                                                round(mean/train[target_col].mean(), 2)))
del mean

In [ ]:
plt.figure(figsize=(10,3.5))

plt.subplot(122)
plt.title("When user gives at least one show a 0 rating")
plt.xlabel('Average scores given')
plt.hist((train.loc[(train==0).max(axis=1)]==0).mean(axis=1))

pyplot.subplot(121)
plt.title("In general")
plt.xlabel('Average scores given')
plt.hist(train.mean(axis=1))

pyplot.show()

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

log_mod = LogisticRegression()

log_mod.fit(train.drop(target_col, axis=1).apply(lambda col:col.fillna(col.median())).values, train[target_col]>0)

log_pred = log_mod.predict_proba(test.drop(target_col, axis=1).apply(lambda col:col.fillna(train[col.name].median())).values)[:,1]

In [ ]:
# benchmark purely random guess
mean = test.loc[pd.Series(log_pred, index=test.index).sort_values(ascending=False).head(int(len(test)*target_frac)).index, target_col].mean()
print("Logistic regression achieves an average score of {} for a lift of {}".format(round(mean, 2), 
                                                                                round(mean/train[target_col].mean(), 2)))
del mean

### Collaborative Filtering

In [ ]:
from itertools import combinations
from sklearn.metrics import pairwise_distances

#store = {}

overall = pd.Series(index=test.index)

for i in range(1,len(train.columns)):
    for combine in combinations(train.columns.drop(target_col), i):
        #print(combine, sum(~train[list(combine)].isna().max(axis=1)))
        #store[combine] = train.loc[~train[list(combine)].isna().max(axis=1), target_col].mean()
        
        train_subset = train.loc[~train[list(combine)].isna().max(axis=1)]
        
        train_subset.groupby(train_subset.columns.drop(target_col))
        test_subset = test.loc[~test[list(combine)].isna().max(axis=1)]

        users = pd.Series(train.loc[pd.DataFrame(pairwise_distances(train_subset, test_subset, metric='euclidean'), 
                                         index=train_subset.index,
                                            columns=test_subset.index).idxmin().values, 
                                    target_col].values,
                          index=test_subset.index)
        overall.loc[users.index] = users

In [ ]:
train_subset.groupby(list(train.columns.drop(target_col)))[target_col].mean().to_frame()

In [ ]:
# benchmark collaborative filtering
mean = test.loc[overall.sort_values(ascending=False).head(int(len(test)*target_frac)).index,
         target_col].mean()
print("Collaborative filtering achieves an average score of {} for a lift of {}".format(round(mean, 2), 
                                                                                round(mean/train[target_col].mean(), 2)))
del mean